# Predict NFL Touchdowns

***

Here in this section I am going to attempt to make a model that will predict who will score nfl touchdowns next season.

In [30]:
# import required libaries
import pandas as pd
import glob
import os
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from ipywidgets import interact, interactive, fixed, Layout, Dropdown


In [2]:
# Read in multiple csv with same headings to 1 dataframe
# https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
path = '.\Data\Pred_data'
all_files = glob.glob(os.path.join(path, "*.csv"))

df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

C:\Users\elyn\AppData\Local\Temp\ipykernel_20412\756877225.py:6: DtypeWarning: Columns (179,180,182,183,189,190,193,194,197,198,203,204,205,206,213,214,218,219,220,222,224,226,248,249,253,254,255,260,262,263,266,267,268,269,283,284,301) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
C:\Users\elyn\AppData\Local\Temp\ipykernel_20412\756877225.py:6: DtypeWarning: Columns (179,180,182,183,189,190,193,194,197,198,203,204,205,206,218,219,220,233,234,235,236,237,238,248,249,253,254,255,260,262,263,283,284,301) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
C:\Users\elyn\AppData\Local\Temp\ipykernel_20412\756877225.py:6: DtypeWarning: Columns (179,180,182,183,189,190,193,194,197,198,203,204,205,206,218,219,220,222,224,226,243,244,245,253,254,255,260,262,263,283,284,301) have mixed types. Specify d

In [3]:
# reset the index as each csv will have its own
df = df.reset_index(drop=True)

In [4]:
# Take a peek
df.sample(20)

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
78165,3872,2021_12_ATL_JAX,2021112803,JAX,ATL,REG,12,JAX,home,ATL,...,0,0,1.045156,0.093155,3.428529,1.0,0.996073,0.996073,0.991442,0.855821
38272,3115,2020_15_CLE_NYG,2020122009,NYG,CLE,REG,15,NYG,home,CLE,...,0,1,0.205995,0.410963,3.516259,2.0,0.984672,0.270272,0.933438,6.656247
123913,3506,2022_10_DAL_GB,2022111309,GB,DAL,REG,10,GB,home,DAL,...,1,0,-0.223389,NaN,NaN,NaN,NaN,NaN,0.588834,-58.883423
18628,5121,2020_07_SEA_ARI,2020102508,ARI,SEA,REG,7,SEA,away,ARI,...,0,0,0.481734,0.114162,3.014035,1.0,1.000000,0.992998,0.660932,33.906811
98991,2557,2021_21_SF_LA,2022013001,LA,SF,POST,21,SF,away,LA,...,0,1,2.746946,NaN,NaN,NaN,NaN,NaN,0.847609,15.239066
72864,3859,2021_09_TEN_LA,2021110711,LA,TEN,REG,9,LA,home,TEN,...,1,1,0.707411,0.964645,11.151156,10.0,0.515984,0.234816,0.881546,11.845422
109319,126,2022_04_MIA_CIN,2022092900,CIN,MIA,REG,4,CIN,home,MIA,...,0,1,0.569643,0.999514,7.657624,6.0,0.484590,0.225526,0.479934,52.006626
23655,3964,2020_09_NYG_WAS,2020110807,WAS,NYG,REG,9,WAS,home,NYG,...,0,0,-3.983204,1.418594,7.827640,7.0,0.360385,0.278089,0.979373,2.062666
29799,4069,2020_12_CHI_GB,2020112911,GB,CHI,REG,12,GB,home,CHI,...,0,1,1.409459,NaN,NaN,NaN,NaN,NaN,0.059376,-5.937551
37175,3068,2020_14_PIT_BUF,2020121313,BUF,PIT,REG,14,PIT,away,BUF,...,0,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# See the shape of the dataframe
print(df.shape)

(149373, 372)


# Grouping the data

***

As you can see this is a fairly large dataframe, in comparison to what I have been using up to now it is enormous. I will not be using all the data here to create my model so I will now begin cleaning and grouping the data I need. I will look through the column titles and find the colums with the data I want to analyses to see if there is a corrolation between that data and touchdown passess.

In [6]:
# list all columns in the dataframe 
df.columns.values

array(['play_id', 'game_id', 'old_game_id', 'home_team', 'away_team',
       'season_type', 'week', 'posteam', 'posteam_type', 'defteam',
       'side_of_field', 'yardline_100', 'game_date',
       'quarter_seconds_remaining', 'half_seconds_remaining',
       'game_seconds_remaining', 'game_half', 'quarter_end', 'drive',
       'sp', 'qtr', 'down', 'goal_to_go', 'time', 'yrdln', 'ydstogo',
       'ydsnet', 'desc', 'play_type', 'yards_gained', 'shotgun',
       'no_huddle', 'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble',
       'pass_length', 'pass_location', 'air_yards', 'yards_after_catch',
       'run_location', 'run_gap', 'field_goal_result', 'kick_distance',
       'extra_point_result', 'two_point_conv_result',
       'home_timeouts_remaining', 'away_timeouts_remaining', 'timeout',
       'timeout_team', 'td_team', 'td_player_name', 'td_player_id',
       'posteam_timeouts_remaining', 'defteam_timeouts_remaining',
       'total_home_score', 'total_away_score', 'posteam_score'

In [34]:
# Quarterback statistics to consider
qb_stats = [
    'season', 'passer_id', 'passer', 'pass', 
    'complete_pass', 'interception', 
    'sack', 'yards_gained', 'touchdown', 'qb_dropback', 'wind'
]

In [35]:
# Grouping criteria
groupby_stats = [
    'season', 'passer_id', 'passer'
]

Aggregating statistics in the context of data analysis refers to the process of summarizing and condensing data values within groups.  
The selected subset of columns are grouped by the columns listed in groupby_stats. It groups the data into distinct groups based on unique combinations of values in the specified columns.

In [36]:
# group by and aggregate by sum 
qb_df = (df
         .loc[:, qb_stats]
         .groupby(groupby_stats, 
                  as_index=False)
         .sum())

# random sample of 10 player-seasons 
qb_df.sample(10)

,season,passer_id,passer,pass,complete_pass,interception,sack,yards_gained,touchdown,qb_dropback,wind
293,2022,00-0033662,C.Rush,180,94.0,3.0,7.0,1010.0,5.0,170.0,7.0
93,2020,00-0035040,D.Blough,14,6.0,1.0,1.0,57.0,0.0,12.0,0.0
183,2021,00-0033662,C.Rush,57,30.0,1.0,3.0,405.0,3.0,51.0,0.0
20,2020,00-0027939,C.Newton,440,242.0,10.0,31.0,2649.0,10.0,418.0,2624.0
131,2021,00-0026498,M.Stafford,830,502.0,20.0,37.0,5887.0,54.0,794.0,0.0
297,2022,00-0033893,D.Cook,1,0.0,0.0,1.0,-5.0,0.0,1.0,0.0
356,2022,00-0038108,B.Zappe,109,65.0,3.0,6.0,746.0,5.0,99.0,0.0
112,2020,00-0036312,J.Luton,123,60.0,6.0,7.0,585.0,3.0,119.0,1622.0
318,2022,00-0035228,K.Murray,465,259.0,7.0,25.0,2389.0,17.0,443.0,0.0
207,2021,00-0034899,J.Wolford,5,1.0,1.0,1.0,-3.0,0.0,5.0,0.0


In [39]:

# List of variables to analyze correlation with touchdowns
variables_to_analyze = ['yards_gained', 'complete_pass', 'pass', 'interception', 'sack', 'qb_dropback', 'wind']

# Define layout for centering the dropdown
centered_layout = Layout(display='flex', justify_content='center')

@interact(variable=variables_to_analyze, width=(6, 20), height=(4, 12), layout=centered_layout)
def plot_correlation(variable, width, height):
    plt.figure(figsize=(width, height))  # Set figsize
    
    # Set scatter plot properties (dots red)
    scatter_kws = {'color': 'red'}
    # Set regression line properties (line blue)
    line_kws = {'color': 'blue'}
    
    sns.regplot(data=qb_df, x='touchdown', y=variable, scatter_kws=scatter_kws, line_kws=line_kws)
    plt.title(f"Correlation between Touchdowns and {variable}")
    plt.xlabel('Touchdowns')
    plt.ylabel(variable)
    plt.show()

interactive(children=(Dropdown(description='variable', options=('yards_gained', 'complete_pass', 'pass', 'inte…

In Seaborn, the sns.regplot() function is used to create a scatter plot with a regression line fit through the data points. It allows the user to visually explore the relationship between two variables and estimate the linear relationship between them. The main purpose of regplot is to provide insights into the correlation and trend between two variables, and thankfully it does the linear regression function and fits the line for me.

Each dot corresponds to a specific data instance (e.g., a player-season in the dataset).  
 - The x-coordinate of each dot represents the number of touchdowns scored by the player in that season.  
 - The y-coordinate of each dot represents the value of the selected variable (e.g., yards gained, completed passes, etc.) for the same player-season.  
 
The blue line is a linear regression line that best fits the scatter plot's data points.  
It represents the general trend or relationship between the two variables: touchdowns and the selected variable.
 - The slope of the line indicates the change in the selected variable's value as the number of touchdowns changes.
 - If the line has a positive slope, it suggests that higher touchdowns are associated with higher values of the selected variable. Conversely, a negative slope indicates an inverse relationship.

The shaded blue section around the regression line represents the range within which the actual regression line is likely to fall with a certain level of confidence.  
 - It indicates the uncertainty in the estimated regression line due to the variability in the data points.
 - The wider the shaded blue section, the greater the uncertainty in the regression line's position. Conversely, a narrower shaded section indicates higher confidence in the regression line's position.